# **Modeling**

### 1. Tổng quan & Chiến lược tiếp cận

Sau khi hoàn tất quá trình Khám phá dữ liệu (EDA) và Tiền xử lý (Preprocessing), chúng ta đã có một bộ dữ liệu sạch, được mã hóa và bổ sung các đặc trưng tương tác mạnh mẽ (như `is_startup_veteran`, `brain_drain_risk`...).

Mục tiêu của chương này là xây dựng các mô hình Máy học (Machine Learning) để giải quyết bài toán **Phân loại nhị phân (Binary Classification)**: Dự đoán xác suất một ứng viên sẽ thay đổi công việc (`target = 1`) hay ở lại (`target = 0`).

Chiến lược Modeling được thực hiện theo nguyên tắc **"Từ đơn giản đến phức tạp"**, bao gồm 3 giai đoạn:

1.  **Thiết lập Baseline (Mô hình cơ sở):** Sử dụng **Logistic Regression** để đánh giá hiệu quả của các biến đầu vào và thiết lập mức chuẩn so sánh.
2.  **Thử nghiệm mô hình phi tuyến:** Sử dụng **Random Forest** 
3.  **Tối ưu hóa với mô hình phi tuyến:** Sử dụng **XGBoost** để nắm bắt các mối quan hệ phức tạp và các điểm gãy (splits) mà hai mô hình trên có thể bỏ sót.

### 2. Tiêu chí đánh giá (Evaluation Metrics)

Do bộ dữ liệu có sự mất cân bằng nhẹ (tỷ lệ `target=1` chiếm ~25%), việc chỉ sử dụng độ chính xác (**Accuracy**) có thể gây hiểu nhầm. Chúng ta sẽ tập trung vào các chỉ số sau:

* **Recall:** Đây là chỉ số quan trọng nhất về mặt quản trị rủi ro. Chúng ta muốn mô hình "bắt" được tối đa những người có ý định nghỉ việc để HR kịp thời can thiệp.
* **F1-Score:** Thước đo hài hòa giữa Precision và Recall, giúp đánh giá tổng quát hiệu năng của mô hình trên nhóm thiểu số.
* **ROC-AUC:** Đánh giá khả năng phân loại của mô hình ở các ngưỡng (threshold) khác nhau.

### 3. Quy trình Huấn luyện (Training Pipeline)

Quá trình huấn luyện sẽ tuân thủ các bước:
1.  **Chia tập dữ liệu (Train-Test Split):** Tỷ lệ 80% Train - 20% Test để đảm bảo mô hình được đánh giá khách quan trên dữ liệu chưa từng thấy.
2.  **Tinh chỉnh tham số (Hyperparameter Tuning):** Sử dụng `GridSearchCV` để tìm ra bộ tham số tối ưu nhất cho từng thuật toán (ví dụ: tìm `k` cho KNN, `n_estimators` cho Random Forest).
3.  **Phân tích kết quả:** So sánh hiệu năng giữa các mô hình và phân tích lí do nguyên nhân vì sao lại như vậy.
---

## **Setup & Import**

- Khởi tạo đường dẫn project.
- Import các hàm/mô hình đã cài trong `src/`.
- Set random seed để kết quả tái lập.


In [1]:
import os
import sys
import numpy as np

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from src.models import (
    load_xy_from_clean_csv,
    train_val_test_split,
    roc_auc_score,
    LogisticRegression,
    RandomForest,
    XGBoost,
    evaluate_binary_classification,
    find_best_threshold,
)

np.set_printoptions(suppress=True, linewidth=120)
RANDOM_STATE = 23120329
THRESHOLD_DEFAULT = 0.5

## **Load & chuẩn bị dữ liệu**

- Đọc file CSV đã được xử lý sạch .
- Tách `X` (feature) và `y` (target).
- Chia dữ liệu: Từ tập dữ liệu sạch ban đầu, ta chia thành 3 phần:
    * **Train (~60%):** dùng để cập nhật trọng số mô hình.
    * **Validation (~20%):** dùng để:
        * Theo dõi val loss trong quá trình training.
        * Áp dụng early stopping.
        * Chọn threshold tối ưu.
    * **Test (~20%):** chỉ sử dụng một lần ở cuối để đánh giá mô hình sau khi mọi quyết định về kiến trúc, hyperparameter và threshold đã được cố định.


In [2]:
CLEAN_CSV_PATH = os.path.join(PROJECT_ROOT, "data/processed", "aug_train.csv")

X, y, feature_names = load_xy_from_clean_csv(CLEAN_CSV_PATH, target_col="target")

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Số feature:", len(feature_names))

X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
    X,
    y,
    val_ratio=0.2,
    test_ratio=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

print("Train:", X_train.shape)
print("Val  :", X_val.shape)
print("Test :", X_test.shape)

X shape: (19158, 28)
y shape: (19158,)
Số feature: 28
Train: (11496, 28)
Val  : (3831, 28)
Test : (3831, 28)


## **Mô hình 1: Logistic Regression**

**Lý do chọn:**
* Đây là mô hình chuẩn, đơn giản, dễ diễn giải: mỗi hệ số tương ứng “tác động” của một feature đến log-odds đổi việc.
* Logistic Regression thường được dùng làm baseline cho các bài toán churn/attrition, nên rất phù hợp với bài toán HR này.

**Thiết lập:**
* Dùng phiên bản Logistic Regression tự cài bằng NumPy:
    * Mini-batch gradient descent.
    * Optimizer Adam.
    * Regularization L2 nhỏ để giảm overfitting.
* Dữ liệu được chuẩn hóa (standardization) trên train, rồi áp dụng cho val/test.
* Trong fit, mỗi epoch:
    * Duyệt qua các mini-batch.
    * Cập nhật trọng số bằng Adam.
* Sau epoch, tính:
    * Train loss (binary cross-entropy trên train).
    * Val loss (nếu có val).
* Thanh tiến trình (`tqdm`) hiển thị epoch hiện tại, train loss và val loss.

**Early stopping:**
* Dùng val loss làm tiêu chí:
    * Nếu val loss không giảm sau `patience` epoch liên tiếp, dừng sớm.
    * Giữ lại trọng số tốt nhất quan sát được (best val loss).
* Lý do: Logistic là mô hình đơn giản nhưng vẫn có thể overfit nếu số epoch quá lớn, đặc biệt với dữ liệu không cân bằng.

**Đánh giá:**
* Trên validation:
    * Tính accuracy, precision, recall, F1, confusion matrix, roc_auc.
    * Threshold mặc định: THRESHOLD_DEFAULT.
* Sau khi hài lòng với setting, dùng cùng mô hình và threshold THRESHOLD_DEFAULT để đánh giá trên test.

**Vai trò:**
Logistic Regression cung cấp một baseline tuyến tính:
* Cho thấy mức performance “đơn giản nhất” ta đạt được.
* Là mốc để so sánh xem Random Forest và XGBoost có thật sự đem lại lợi ích hay chỉ phức tạp hơn.


### **Trainning**
Đoạn code thực hiện **Grid Search** thủ công để tìm bộ tham số tốt nhất cho mô hình Logistic Regression:

1.  **Thiết lập không gian tìm kiếm:**
    * `Learning Rate`: [0.01, 0.05]
    * `L2 Regularization`: [0.0, 0.001]
    * `Batch Size`: [128, 256]

2.  **Quy trình thực hiện:**
    * Duyệt qua từng tổ hợp tham số.
    * Huấn luyện mô hình với cơ chế **Early Stopping** (dừng sớm nếu không cải thiện sau 30 epochs) và tối ưu hóa bằng **Adam**.
    * Đánh giá mô hình trên tập **Validation**.

3.  **Lựa chọn mô hình:**
    * Sử dụng **F1-Score** làm tiêu chí quyết định.
    * Lưu lại bộ tham số và mô hình có F1-Score cao nhất trên tập Validation.

In [3]:
# Grid search Logistic Regression
lr_list = [0.01, 0.05]
l2_list = [0.0, 0.001]
batch_list = [128, 256]

best_logreg = None
best_logreg_f1 = -1.0
best_params = None
best_threshold = None

print("Thực hiện Grid Search cho Logistic Regression...\n")

for lr_ in lr_list:
    for l2_ in l2_list:
        for batch_ in batch_list:
            print(f"[lr={lr_}, l2={l2_}, batch={batch_}]")
            
            model = LogisticRegression(
                lr=lr_, 
                n_epochs=100, 
                batch_size=batch_, 
                l2=l2_, 
                l1=0.0, 
                optimizer="adam", 
                lr_decay=1e-3, 
                early_stopping=True, 
                patience=30, 
                random_state=RANDOM_STATE
            )

            model.fit(X_train, y_train, X_val=X_val, y_val=y_val)
            
            y_val_prob_tmp = model.predict_proba(X_val)

            t_tmp, f1_tmp = find_best_threshold(y_val, y_val_prob_tmp, n_points=200)

            metrics_tmp = evaluate_binary_classification(
                y_true=y_val,
                y_prob=y_val_prob_tmp,
                threshold=t_tmp
            )

            print(f"  F1-score Validation = {metrics_tmp['f1']:.4f}\n")
            
            if metrics_tmp["f1"] > best_logreg_f1:
                best_logreg = model
                best_logreg_f1 = metrics_tmp["f1"]
                best_params = (lr_, l2_, batch_, metrics_tmp)
                best_threshold = t_tmp   

m = best_params[3]

print("Bộ tham số tốt nhất:")
print(f"Best learning rate = {best_params[0]}")
print(f"Best l2            = {best_params[1]}")
print(f"Best batch         = {best_params[2]}")
print(f"Best threshold     = {best_threshold:.4f}")

print("\nMetrics:")
print(f"  Accuracy         : {m['accuracy']:.4f}")
print(f"  Precision        : {m['precision']:.4f}")
print(f"  Recall           : {m['recall']:.4f}")
print(f"  F1-score         : {m['f1']:.4f}")
print(f"  Threshold        : {best_threshold:.4f}")  
print(f"  Confusion Matrix :\n{m['confusion_matrix']}")

Thực hiện Grid Search cho Logistic Regression...

[lr=0.01, l2=0.0, batch=128]


LogisticRegression:  54%|█████▍    | 54/100 [00:00<00:00, 162.89it/s, train=0.4609, val=0.4584]


[Early stopping] quá trình huấn luyện dừng lại tại epoch 55 | best_loss = 0.4575
  F1-score Validation = 0.5929

[lr=0.01, l2=0.0, batch=256]


LogisticRegression:  47%|████▋     | 47/100 [00:00<00:00, 198.23it/s, train=0.4612, val=0.4576]


[Early stopping] quá trình huấn luyện dừng lại tại epoch 48 | best_loss = 0.4576
  F1-score Validation = 0.5923

[lr=0.01, l2=0.001, batch=128]


...

[Early stopping] quá trình huấn luyện dừng lại tại epoch 37 | best_loss = 0.4576
  F1-score Validation = 0.5939

[lr=0.05, l2=0.001, batch=128]


LogisticRegression:  47%|████▋     | 47/100 [00:00<00:00, 145.17it/s, train=0.4646, val=0.4597]


[Early stopping] quá trình huấn luyện dừng lại tại epoch 48 | best_loss = 0.4591
  F1-score Validation = 0.5869

[lr=0.05, l2=0.001, batch=256]


LogisticRegression:  36%|███▌      | 36/100 [00:00<00:00, 188.23it/s, train=0.4653, val=0.4616]

[Early stopping] quá trình huấn luyện dừng lại tại epoch 37 | best_loss = 0.4589
  F1-score Validation = 0.5889

Bộ tham số tốt nhất:
Best learning rate = 0.05
Best l2            = 0.0
Best batch         = 256
Best threshold     = 0.2663

Metrics:
  Accuracy         : 0.7552
  Precision        : 0.4996
  Recall           : 0.7321
  F1-score         : 0.5939
  Threshold        : 0.2663
  Confusion Matrix :
[[2207  687]
 [ 251  686]]


#### **Nhận xét quá trình huấn luyện Logistic Regression**

**1. Khả năng hội tụ & Ổn định:**

Trong quá trình Grid Search, Logistic Regression thể hiện khả năng hội tụ rất nhanh và ổn định với mọi cấu hình:
* **Early Stopping hiệu quả:** Tất cả các mô hình đều dừng sớm sau khoảng **35–60 epoch**.
* **Không Overfitting:** Loss giảm đều đặn, chứng tỏ mô hình học tốt mà không bị khớp quá kỹ vào tập train.
* **Độ nhạy:** Mô hình khá bền vững với thay đổi nhỏ của batch size hay learning rate, nhưng phản ứng rõ rệt với Regularization.

**2. Về Bộ tham số tối ưu, cấu hình tốt nhất (Best Params):**
* **Regularization (L2 = 0.0):** Việc L2 = 0.0 cho kết quả tốt hơn L2 = 0.001 chứng tỏ dữ liệu đã được xử lý tiền kỳ rất tốt (sạch nhiễu, chuẩn hóa, one-hot). Việc thêm Regularization lúc này là không cần thiết và thậm chí làm giảm tính linh hoạt của mô hình.
* **Learning Rate (0.05):** Hiệu quả hơn 0.01 nhờ tốc độ cập nhật trọng số nhanh hơn mà vẫn duy trì được sự ổn định.
* **Batch Size (256):** Cho kết quả tốt nhất nhờ tạo ra gradient mượt mà và tối ưu hóa ổn định hơn so với các batch nhỏ.

### **Đánh giá với test set**

Quy trình đánh giá được thực hiện theo chiến lược 2 giai đoạn để đảm bảo tính khách quan:

1.  **Dự báo xác suất:**
    * Tính xác suất (`predict_proba`) cho tập Validation và Test thay vì chỉ dự đoán nhãn cứng (0/1).

2.  **Đánh giá hiệu năng (Final Evaluation):**
    * Áp dụng ngưỡng tối ưu từ Validation lên tập **Test** (tránh data leakage).
    * Xuất các chỉ số quan trọng (Recall, Precision, F1, Confusion Matrix, ROC-AUC) để kết luận năng lực thực tế của mô hình.

In [4]:
y_val_prob_log  = best_logreg.predict_proba(X_val)
y_test_prob_log = best_logreg.predict_proba(X_test)

best_t_log = best_threshold

metrics_log_val  = evaluate_binary_classification(y_val,  y_val_prob_log,  threshold=best_t_log)
metrics_log_test = evaluate_binary_classification(y_test, y_test_prob_log, threshold=best_t_log)

auc_val_log  = roc_auc_score(y_val,  y_val_prob_log)
auc_test_log = roc_auc_score(y_test, y_test_prob_log)

print(f"Best threshold Logistic theo F1-score: {best_t_log:.3f}")

print("\n" + "="*60)
print("\t   Logistic Regression – Validation vs Test")
print("="*60)

print(f"{'Metric':15s} | {'Validation':12s} | {'Test':12s}")
print("-"*60)

for k in metrics_log_val.keys():
    if k == "confusion_matrix":
        continue
    
    v_val  = metrics_log_val[k]
    v_test = metrics_log_test[k]

    if isinstance(v_val, float):
        print(f"{k:15s} | {v_val:12.4f} | {v_test:12.4f}")
    else:
        print(f"{k:15s} | {str(v_val):12s} | {str(v_test):12s}")

print(f"{'roc_auc':15s} | {auc_val_log:12.4f} | {auc_test_log:12.4f}")

print("-"*60)
print("\nConfusion matrix (Validation):")
print(metrics_log_val["confusion_matrix"])

print("\nConfusion matrix (Test):")
print(metrics_log_test["confusion_matrix"])

Best threshold Logistic theo F1-score: 0.266

	   Logistic Regression – Validation vs Test
Metric          | Validation   | Test        
------------------------------------------------------------
accuracy        |       0.7552 |       0.7648
precision       |       0.4996 |       0.5267
recall          |       0.7321 |       0.7392
f1              |       0.5939 |       0.6151
threshold       |       0.2663 |       0.2663
roc_auc         |       0.7877 |       0.7954
------------------------------------------------------------

Confusion matrix (Validation):
[[2207  687]
 [ 251  686]]

Confusion matrix (Test):
[[2210  647]
 [ 254  720]]


#### **Nhận xét**

Dựa trên kết quả thực nghiệm trên tập Validation và Test, ta có những nhận xét sau về mô hình cơ sở (Baseline):

  1. Độ ổn định giữa Validation và Test cho thấy một tín hiệu rất tốt

     So sánh hiệu năng giữa hai tập dữ liệu:

      | Metric | Val | Test | Nhận xét |
      | :--- | :---: | :---: | :--- |
      | **Accuracy** | 0.7552 | **0.7648** | Tăng nhẹ  |
      | **Precision** | 0.4996 | **0.5267** | Tăng   |
      | **Recall** | 0.7321 | **0.7392** | Tăng nhẹ |
      | **F1-Score** | 0.5939 | **0.6151** | Tăng   |
      | **ROC-AUC** | 0.7877 | **0.7954** | Tăng nhẹ |

**Điểm mạnh:** Mô hình không bị **Overfitting**. Hiệu năng trên Test nhỉnh hơn Validation chứng tỏ việc chia dữ liệu hợp lý và mô hình có khả năng tổng quát hóa tốt.

  2. Chiến lược "Thà bắt nhầm còn hơn bỏ sót"
    * **Precision (~0.53):** Dự đoán 100 người nghỉ thì đúng khoảng 53 người.
    * **Recall (~0.74):** Bắt được **74%** tổng số người thực sự muốn nghỉ việc.

     **Phù hợp đặc thù HR:** Ưu tiên cao nhất là **không bỏ sót** nhân tài có nguy cơ rời đi (High Recall). Tỷ lệ báo động giả cao hơn một chút là chấp nhận được trong bài toán sàng lọc rủi ro.

  3. Phân tích Ma trận nhầm lẫn (Confusion Matrix)
    Kết quả trên tập Test:
     ```text
     [[2210  647]
     [ 254  720]]
     ```
     * Điểm sáng (Low False Negative): Chỉ có 254 trường hợp nghỉ việc bị bỏ sót, thấp hơn nhiều so với số lượng 720 trường hợp được phát hiện chính xác (True Positive). Đây là thành công lớn nhất của mô hình trong việc cảnh báo sớm.

     * Điểm trừ (High False Positive): Có 647 người bị báo nhầm là nghỉ việc (thực tế ở lại). Điều này gây lãng phí tài nguyên giữ chân, nhưng là sự đánh đổi cần thiết để đạt được Recall cao.

**Kết luận:** Logistic Regression là một Baseline mạnh, hoạt động ổn định và hoàn thành tốt nhiệm vụ khoanh vùng nhóm rủi ro cao.

---

## **Mô hình 2: Random Forest**

**Lý do chọn:**
* Quá trình EDA cho thấy mối quan hệ giữa nhiều biến với target không tuyến tính:
    * Ví dụ các ngưỡng kinh nghiệm, số năm từ lần đổi việc gần nhất, thời gian training,… thường tạo ra “điểm gãy”.
* Random Forest là mô hình tree-based:
    * Mỗi cây quyết định (CART) chia dữ liệu theo threshold trên một feature.
    * Nhiều cây được train trên các bootstrap sample + random subset feature → giảm variance.

**Thiết lập:**
* Cài đặt một DecisionTree đơn giản:
    * Split theo Gini impurity.
    * Dừng theo `max_depth`, `min_samples_split`, node thuần.
* RandomForest:
    * Số cây `n_estimators` (ví dụ 100).
    * `max_depth` để khống chế độ phức tạp mỗi cây.
    * `max_features="sqrt"` để mỗi cây chỉ nhìn thấy một phần feature → tăng đa dạng.
* Train với progress bar hiển thị quá trình build từng cây.
* Khi predict:
    * Mỗi cây đưa ra một vote (0/1).
    * Xác suất lớp 1 ≈ tỷ lệ số cây vote 1.

**Đánh giá:**
* Dùng xác suất này để tính metric:
    * Validation: tính accuracy, precision, recall, F1 (threshold mặc định 0.5).
    * Test: dùng cùng threshold để đánh giá cuối.

**Lợi ích & quan sát:**
* Random Forest cho phép mô hình “uốn cong” decision boundary:
    * Bắt được tương tác giữa các feature (ví dụ experience + last_new_job + training_hours).
* Trong thực nghiệm thường:
    * F1 và/hoặc Recall cao hơn Logistic.
    * Điều này cho thấy việc dùng cây quyết định đúng với giả thiết EDA: quan hệ không đơn thuần tuyến tính.

### **Training**

Thực hiện tìm kiếm bộ tham số tối ưu (**Grid Search**) cho mô hình Random Forest dựa trên **F1-score** trên tập Validation:

1.  **Không gian tìm kiếm:**
    * `n_estimators`: [80, 100] (Số lượng cây).
    * `max_depth`: [5, 6] (Độ sâu tối đa của cây - kiểm soát độ phức tạp).
    * `min_samples_split`: [5, 10] (Số lượng mẫu tối thiểu để chia nút - chống overfitting).

2.  **Quy trình:**
    * Duyệt qua từng tổ hợp tham số.
    * Huấn luyện mô hình trên `X_train`, `y_train`.
    * Dự báo và đánh giá trên tập `X_val` (sử dụng ngưỡng mặc định).

3.  **Kết quả:**
    * Lưu lại mô hình (`best_rf`) và bộ tham số có **F1-score cao nhất**.
    * Xuất ra các chỉ số đánh giá chi tiết (Accuracy, Precision, Recall, F1) của mô hình tốt nhất.

In [5]:
n_estimators_list = [80, 100]
max_depth_list    = [5, 6]
min_samples_list  = [5, 10]

best_rf = None
best_rf_f1 = -1.0
best_rf_params = None
best_rf_threshold = None

print("Thực hiện Grid Search cho Random Forest...\n")

for n_est in n_estimators_list:
    for depth in max_depth_list:
        for min_s in min_samples_list:
            print(f"[n_estimators={n_est}, max_depth={depth}, min_samples_split={min_s}]")

            rf_tmp = RandomForest(
                n_estimators=n_est,
                max_depth=depth,
                max_features="sqrt",
                min_samples_split=min_s,
                random_state=RANDOM_STATE,
            )
            rf_tmp.fit(X_train, y_train, X_val=X_val, y_val=y_val)

            y_val_prob_tmp = rf_tmp.predict_proba(X_val)

            t_tmp, f1_tmp = find_best_threshold(y_val, y_val_prob_tmp, n_points=200)

            metrics_tmp = evaluate_binary_classification(
                y_true=y_val,
                y_prob=y_val_prob_tmp,
                threshold=t_tmp,
            )

            print(f"  F1-score validation = {metrics_tmp['f1']:.4f}\n")
            
            if metrics_tmp["f1"] > best_rf_f1:
                best_rf_f1 = metrics_tmp["f1"]
                best_rf = rf_tmp
                best_rf_params = (n_est, depth, min_s, metrics_tmp)
                best_rf_threshold = t_tmp

print("Bộ tham số tốt nhất")
print("  n_estimators      =", best_rf_params[0])
print("  max_depth         =", best_rf_params[1])
print("  min_samples_split =", best_rf_params[2])
print(f"  best_threshold    = {best_rf_threshold:.4f}")

m_best = best_rf_params[3]
print("\nMetrics tốt nhất trên Validation:")
print(f"  Accuracy         : {m_best['accuracy']:.4f}")
print(f"  Precision        : {m_best['precision']:.4f}")
print(f"  Recall           : {m_best['recall']:.4f}")
print(f"  F1-score         : {m_best['f1']:.4f}")
print(f"  Threshold        : {best_rf_threshold:.4f}")
print(f"  Confusion Matrix :\n{m_best['confusion_matrix']}")

Thực hiện Grid Search cho Random Forest...

[n_estimators=80, max_depth=5, min_samples_split=5]


RandomForest: 100%|██████████| 80/80 [02:07<00:00,  1.60s/it, train=2.1860, val=2.2913]


  F1-score validation = 0.5949

[n_estimators=80, max_depth=5, min_samples_split=10]


RandomForest: 100%|██████████| 80/80 [02:01<00:00,  1.51s/it, train=2.1807, val=2.2907]


  F1-score validation = 0.5948

[n_estimators=80, max_depth=6, min_samples_split=5]


...

  F1-score validation = 0.6012

[n_estimators=100, max_depth=6, min_samples_split=5]


RandomForest: 100%|██████████| 100/100 [02:29<00:00,  1.49s/it, train=1.7585, val=1.9891]


  F1-score validation = 0.5983

[n_estimators=100, max_depth=6, min_samples_split=10]


RandomForest: 100%|██████████| 100/100 [03:02<00:00,  1.83s/it, train=1.7584, val=1.9889]


  F1-score validation = 0.5980

Bộ tham số tốt nhất
  n_estimators      = 100
  max_depth         = 5
  min_samples_split = 5
  best_threshold    = 0.0302

Metrics tốt nhất trên Validation:
  Accuracy         : 0.7794
  Precision        : 0.5388
  Recall           : 0.6820
  F1-score         : 0.6020
  Threshold        : 0.0302
  Confusion Matrix :
[[2347  547]
 [ 298  639]]


#### **Nhận xét về quá trình huấn luyện**

Quá trình huấn luyện Random Forest diễn ra ổn định và nhất quán giữa các bộ tham số. Các mô hình với cùng cấu hình độ sâu (`max_depth`) cho thấy hành vi gần giống nhau, cả về loss lẫn F1-score. Điều này cho thấy mô hình **không quá nhạy cảm với `min_samples_split`**, nhưng lại **nhạy cảm với độ sâu và số lượng cây**. Ngoài ra thời gian training lâu hơn khá nhiều so với Logistic Regression vì Random Forest phải xây dựng rất nhiều cây (Decision Tree), và mỗi cây phải thử rất nhiều split trên nhiều feature để tìm điểm chia tốt nhất. Logistic Regression chỉ cần tối ưu một hàm tuyến tính bằng phép nhân ma trận, nên tính toán đơn giản và nhanh hơn rất nhiều.

**Một điểm dễ thấy là:**

* Với **`max_depth = 5`**, mô hình ổn định và đạt F1 cao hơn.
* Khi tăng lên **`max_depth = 6`**, train loss giảm (mô hình phức tạp hơn) nhưng validation loss lại không cải thiện và F1 giảm nhẹ.
    *Điều này cho thấy mô hình bắt đầu “học quá sâu”, nhưng không nắm bắt được thêm thông tin hữu ích từ dữ liệu — một dấu hiệu của **quá khớp (overfitting) nhẹ** theo chiều sâu.*

Trong khi đó, việc tăng số cây từ **80 → 100** consistently giúp cải thiện nhẹ hiệu năng. Điều này đúng với bản chất Random Forest: thêm cây → mô hình ổn định và tổng hợp tín hiệu tốt hơn, đặc biệt trong dữ liệu có nhiều biến one-hot.


**Về phần bộ tham số tối ưu:** Bộ tham số tốt nhất được xác định là:
 * `n_estimators = 100`
 * `max_depth = 5`
 * `min_samples_split = 5`
 * `best_threshold = 0.0302`

    **Giải thích chi tiết:**

    1.  **`n_estimators = 100`**: 100 cây giúp mô hình ổn định và giảm phương sai hơn so với 80 cây.
    2.  **`max_depth = 5`**: Độ sâu này cho thấy mô hình không cần quá phức tạp — sâu hơn (`depth=6`) không mang lại giá trị mà còn làm validation loss tăng.
    3.  **`min_samples_split = 5`**: Tương thích với `depth=5`, cho phép chia nhánh linh hoạt hơn nhưng vẫn giữ được tính khái quát.
    4.  **`best_threshold = 0.0302`**:
        * Điểm quan trọng nhất là threshold rất thấp. Điều này phản ánh xác suất dự đoán từ Random Forest phân bố khá thấp, đặc biệt cho lớp dương.
        * Để tối ưu F1-score, ta phải hạ threshold xuống mức rất nhỏ.
        * *Đây là hành vi thường thấy ở Random Forest với dữ liệu tồn tại nhiều biến one-hot và phân bố lệch — xác suất mô hình thường không được calibrate tốt, nhưng phân hạng vẫn tương đối ổn.*

### **Đánh giá với test set**
Thực hiện quy trình đánh giá chuyên sâu cho mô hình Random Forest đã được tinh chỉnh (`best_rf`):

1.  **Dự báo xác suất:** Tính xác suất (`predict_proba`) trên tập Validation và Test.
2.  **Đánh giá toàn diện:**
    * Áp dụng ngưỡng tối ưu này để đánh giá lại tập Validation và **Test**.
    * Tính thêm chỉ số **ROC-AUC** để đo khả năng phân loại tổng quát.
3.  **So sánh & Báo cáo:**
    * Xuất bảng so sánh trực quan các chỉ số (Accuracy, Precision, Recall, F1, AUC) giữa Validation và Test để kiểm tra độ ổn định (tránh Overfitting).
    * Hiển thị Ma trận nhầm lẫn (Confusion Matrix) để phân tích chi tiết các lỗi dự báo.

In [6]:
y_val_prob_rf  = best_rf.predict_proba(X_val)
y_test_prob_rf = best_rf.predict_proba(X_test)

best_t_rf  = best_rf_threshold
best_f1_rf = best_rf_f1

metrics_rf_val_best  = evaluate_binary_classification(y_val,  y_val_prob_rf,  threshold=best_t_rf)
metrics_rf_test_best = evaluate_binary_classification(y_test, y_test_prob_rf, threshold=best_t_rf)

auc_val_rf  = roc_auc_score(y_val,  y_val_prob_rf)
auc_test_rf = roc_auc_score(y_test, y_test_prob_rf)

print(f"Best threshold (validation) = {best_t_rf:.4f}\n F1-score (validation) = {best_f1_rf:.4f}")

print("\n" + "="*60)
print("             Random Forest – Validation vs Test")
print("="*60)

print(f"{'Metric':15s} | {'Validation':12s} | {'Test':12s}")
print("-"*60)

for k in metrics_rf_val_best.keys():
    if k == "confusion_matrix":
        continue

    v_val  = metrics_rf_val_best[k]
    v_test = metrics_rf_test_best[k]

    if isinstance(v_val, float):
        print(f"{k:15s} | {v_val:12.4f} | {v_test:12.4f}")
    else:
        print(f"{k:15s} | {str(v_val):12s} | {str(v_test):12s}")

print(f"{'best_threshold':15s} | {best_t_rf:12.4f} | {best_t_rf:12.4f}")
print(f"{'roc_auc':15s} | {auc_val_rf:12.4f} | {auc_test_rf:12.4f}")

print("-"*60)
print("\nConfusion matrix (Validation):")
print(metrics_rf_val_best["confusion_matrix"])

print("\nConfusion matrix (Test):")
print(metrics_rf_test_best["confusion_matrix"])

Best threshold (validation) = 0.0302
 F1-score (validation) = 0.6020

             Random Forest – Validation vs Test
Metric          | Validation   | Test        
------------------------------------------------------------
accuracy        |       0.7794 |       0.7802
precision       |       0.5388 |       0.5555
recall          |       0.6820 |       0.6786
f1              |       0.6020 |       0.6109
threshold       |       0.0302 |       0.0302
best_threshold  |       0.0302 |       0.0302
roc_auc         |       0.7790 |       0.7749
------------------------------------------------------------

Confusion matrix (Validation):
[[2347  547]
 [ 298  639]]

Confusion matrix (Test):
[[2328  529]
 [ 313  661]]


#### **Nhận xét**

  1. Độ ổn định giữa Validation và Test rất tốt

     So sánh hiệu năng giữa hai tập dữ liệu cho thấy sự ổn định tuyệt vời:

        | Metric | Val | Test | Nhận xét |
        | :--- | :---: | :---: | :--- |
        | **Accuracy** | 0.7794 | **0.7802** | Giữ nguyên |
        | **Precision** | 0.5388 | **0.5555** | Tăng nhẹ |
        | **Recall** | 0.6820 | **0.6786** | Giảm rất ít |
        | **F1-Score** | 0.6020 | **0.6109** | Tăng nhẹ |
        | **ROC-AUC** | 0.7790 | **0.7749** | Tương đương nhau |

     **Kết luận:** Random Forest **không bị Overfit** và có khả năng tổng quát hóa (generalize) rất tốt trên tập Test. Đây là yếu tố cực kỳ quan trọng đối với các mô hình dữ liệu dạng bảng (tabular).

  2. So sánh với Logistic Regression (Trade-off)
        | Metric | Logistic | Random Forest | Nhận xét |
        | :--- | :---: | :---: | :--- |
        | **Precision** | ~0.50 | **~0.55** | RF tốt hơn (Ít báo động giả hơn) |
        | **Recall** | **~0.73** | ~0.68 | Logistic cao hơn (Bắt được nhiều người nghỉ hơn) |
        | **F1-Score** | **0.615** | 0.611 | Logistic nhỉnh hơn một chút xíu |
        | **AUC** | **0.79** | 0.78 | Gần tương đương |

        **Phân tích:**

        * **Random Forest:** Nghiêng về độ chính xác (**Precision**), giúp giảm bớt các dự báo sai (False Positive).
        * **Logistic Regression:** Nghiêng về độ bao phủ (**Recall**), chấp nhận bắt nhầm để không bỏ sót.

  3. Điểm thú vị: Ngưỡng (Threshold) cực thấp (0.0302)
        Để đạt F1 cao nhất, ngưỡng cắt của RF hạ xuống còn **0.0302**.
        * **Ý nghĩa:** Chỉ cần xác suất > 3% là mô hình đã dự báo "nghỉ việc".
        * **Lý do:** Các mô hình cây (Tree-based) thường trả ra xác suất không được hiệu chỉnh tốt (uncalibrated probabilities), thường tập trung ở mức thấp đối với dữ liệu mất cân bằng. Nếu dùng ngưỡng mặc định 0.5, Recall sẽ cực tệ.
        * **Kết luận:** Dù xác suất thấp, nhưng khả năng phân loại (ranking) của mô hình vẫn tốt (thể hiện qua AUC ~ 0.78).

  4. Phân tích Ma trận nhầm lẫn (Confusion Matrix)
        Kết quả trên tập Test:

        ```text
        [[2328  529]
        [ 313  661]]
        ```
        * **False Positive giảm mạnh:**
            * So với Logistic Regression (647 ca), Random Forest chỉ có **529 ca**.
            * **Ý nghĩa:** Điều này giúp bộ phận HR đỡ tốn công sức và nguồn lực để "chăm sóc nhầm" những nhân viên vốn dĩ vẫn muốn ở lại (giảm lãng phí).

        * **False Negative tăng lên:**
            * So với Logistic (254 ca), Random Forest bỏ sót **313 ca**.
            * **Ý nghĩa:** Đây là sự đánh đổi (Trade-off) tất yếu. Muốn mô hình có độ chính xác cao hơn (Precision) thì phải chấp nhận rủi ro bỏ sót một số trường hợp nhân viên nghỉ việc.

**Tổng kết: Random Forest vs Logistic Regression**

  1. **Điểm mạnh của Random Forest:**
     * Hoạt động ổn định.
     * **Precision cao hơn:** Dự báo đáng tin cậy hơn.
     * Ít báo động giả (False Positive) hơn.

  2. **Điểm yếu của Random Forest:**
     * **Recall** và **ROC-AUC** thấp hơn một chút so với Logistic.

  3. **Nhận định chung:**
     * **Logistic Regression** vẫn đang là một Baseline cực kỳ mạnh mẽ (nhờ vào quá trình Feature Engineering tốt).
     * **Random Forest** đứng vị trí thứ 2 với ưu thế về sự ổn định và độ chính xác.
     * **Kỳ vọng:** Mô hình tiếp theo (**XGBoost**) sẽ là sự kết hợp điểm mạnh của cả hai: Tăng Precision mà không làm giảm Recall quá nhiều.
---

## **Mô hình 3: XGBoost (Gradient Boosting)**

#### **1. Lý do lựa chọn**
Khác với **Random Forest** sử dụng kỹ thuật Bagging (xây dựng các cây độc lập để giảm phương sai), **XGBoost** áp dụng kỹ thuật **Boosting** nhằm tối ưu hóa độ chính xác:
* **Học tuần tự (Sequential Learning):** Các cây quyết định được xây dựng nối tiếp nhau, trong đó mỗi cây mới sẽ tập trung giải quyết các lỗi sai (residuals) mà chuỗi cây trước đó để lại.
* **Tối ưu hóa bậc hai:** Sử dụng thông tin từ cả Gradient (đạo hàm bậc 1) và Hessian (đạo hàm bậc 2) của hàm mất mát logistic để định hướng việc xây dựng cây, giúp mô hình hội tụ nhanh và chính xác hơn.
* **Hiệu năng vượt trội:** Được công nhận là thuật toán mạnh mẽ nhất (Champion) đối với các bài toán dữ liệu dạng bảng (Tabular Data).

#### **2. Chiến lược thực hiện**
Dự án sử dụng phiên bản XGBoost tự cài đặt bằng **NumPy** để kiểm soát hoàn toàn luồng tính toán:
* **Core:** Xây dựng Class `XGBoost` dựa trên `RegressionTree` kết hợp cơ chế cập nhật trọng số Boosting.
* **Hyperparameter Tuning:** Tinh chỉnh các tham số chủ chốt gồm `n_rounds`, `max_depth`, và `learning_rate` thông qua Grid Search.
* **Threshold Tuning:** Tìm kiếm ngưỡng phân loại tối ưu (Best Threshold) dựa trên chỉ số F1-Score trên tập Validation để cực đại hóa hiệu năng thực tế.

### **Training**

Thực hiện tìm kiếm bộ tham số tối ưu (**Grid Search**) cho mô hình XGBoost nhằm cực đại hóa **F1-score** trên tập Validation:

1.  **Không gian tìm kiếm đa chiều:**
    * `n_rounds`: [80, 120] (Số vòng lặp boosting).
    * `max_depth`: [4, 5] (Độ sâu cây).
    * `learning_rate`: [0.05, 0.1] (Tốc độ học).
    * `subsample` & `colsample`: [0.8, 1.0] (Tỷ lệ mẫu và đặc trưng để chống overfitting).

2.  **Quy trình tối ưu kép:**
    * **Huấn luyện:** Duyệt qua từng tổ hợp tham số, huấn luyện với cơ chế **Early Stopping** (dừng nếu val loss không giảm sau 20 rounds).
    * **Tối ưu ngưỡng:** Với mỗi mô hình, tự động tìm **ngưỡng cắt (threshold)** tốt nhất trên tập Validation thay vì dùng mặc định 0.5.

3.  **Kết quả:**
    * Lưu lại mô hình (`best_xgb`) và bộ tham số mang lại F1-score cao nhất.
    * Ghi nhận ngưỡng tối ưu (`best_xgb_threshold`) để áp dụng đánh giá sau cùng.

In [7]:
# Grid Search XGBoost
n_rounds_list    = [80, 120]
max_depth_list   = [4, 5]
lr_list          = [0.01, 0.05]
subsample_list   = [0.8, 1.0]
colsample_list   = [0.8, 1.0]

best_xgb = None
best_xgb_f1 = -1.0
best_xgb_params = None
best_xgb_threshold = 0.5

print("Đang Grid Search XGBoost...\n")

for n_r in n_rounds_list:
    for depth in max_depth_list:
        for lr_ in lr_list:
            for sub_ in subsample_list:
                for col_ in colsample_list:
                    print(f"[n_rounds={n_r}, max_depth={depth}, lr={lr_}, subsample={sub_}, colsample={col_}]")
                    
                    model = XGBoost(n_rounds=n_r, max_depth=depth, learning_rate=lr_, lambda_reg=1.0, subsample=sub_, colsample_bytree=col_, min_child_weight=1e-2, min_gain_to_split=0.0, early_stopping=True, patience=20, random_state=RANDOM_STATE)
                    model.fit(X_train, y_train, X_val=X_val, y_val=y_val)

                    y_val_prob_tmp = model.predict_proba(X_val)

                    t_tmp, f1_tmp = find_best_threshold(y_val, y_val_prob_tmp, n_points=200)

                    metrics_tmp = evaluate_binary_classification(y_true=y_val,y_prob=y_val_prob_tmp,threshold=t_tmp)

                    print(f"  F1-score validation = {metrics_tmp['f1']:.4f}\n")

                    if f1_tmp > best_xgb_f1:
                        best_xgb_f1 = f1_tmp
                        best_xgb = model
                        best_xgb_threshold = t_tmp
                        best_xgb_params = (n_r, depth, lr_, sub_, col_)

print("\nBộ tham số tốt nhất:")
print(f"  n_rounds      = {best_xgb_params[0]}")
print(f"  max_depth     = {best_xgb_params[1]}")
print(f"  learning_rate = {best_xgb_params[2]}")
print(f"  subsample     = {best_xgb_params[3]}")
print(f"  colsample     = {best_xgb_params[4]}")
print(f"  Best F1(val)  = {best_xgb_f1:.4f}")
print(f"  Best threshold = {best_xgb_threshold:.3f}")

Đang Grid Search XGBoost...

[n_rounds=80, max_depth=4, lr=0.01, subsample=0.8, colsample=0.8]


XGBoost: 100%|██████████| 80/80 [00:16<00:00,  4.95it/s, train=0.5116, val=0.5162]


  F1-score validation = 0.6152

[n_rounds=80, max_depth=4, lr=0.01, subsample=0.8, colsample=1.0]


XGBoost: 100%|██████████| 80/80 [00:20<00:00,  3.85it/s, train=0.5098, val=0.5147]


  F1-score validation = 0.6157

[n_rounds=80, max_depth=4, lr=0.01, subsample=1.0, colsample=0.8]


...

  F1-score validation = 0.6178

[n_rounds=120, max_depth=5, lr=0.05, subsample=1.0, colsample=0.8]


XGBoost: 100%|██████████| 120/120 [00:44<00:00,  2.68it/s, train=0.3982, val=0.4401]


  F1-score validation = 0.6200

[n_rounds=120, max_depth=5, lr=0.05, subsample=1.0, colsample=1.0]


XGBoost:  96%|█████████▌| 115/120 [00:47<00:02,  2.42it/s, train=0.3977, val=0.4407]


[Early stopping] XGBoost dừng tại round 116 | best_val_loss = 0.4406, số cây dùng thực tế = 96
  F1-score validation = 0.6182


Bộ tham số tốt nhất:
  n_rounds      = 120
  max_depth     = 4
  learning_rate = 0.05
  subsample     = 1.0
  colsample     = 0.8
  Best F1(val)  = 0.6261
  Best threshold = 0.286


#### **Nhận xét về quá trình huấn luyện XGBoost**

Quá trình Grid Search của XGBoost cho thấy mô hình phản ứng **ổn định và nhất quán** với tất cả các tổ hợp siêu tham số. Sự khác biệt giữa các cấu hình chủ yếu đến từ `learning_rate` và số vòng boosting, trong khi các tham số như `subsample` và `colsample` chỉ tạo ra khác biệt nhỏ về F1.

**Các xu hướng nổi bật:**
* **Learning Rate:** `0.05` cho F1-score cao hơn đáng kể so với `0.01` ở hầu hết mọi cấu hình.
* **Độ sâu (Max Depth):** Các cấu hình với `max_depth = 4` hoạt động hiệu quả hơn `depth = 5`. Dù độ sâu lớn giúp train loss thấp hơn, nhưng lại không cải thiện (thậm chí làm giảm nhẹ) kết quả trên validation.
* **Số vòng lặp (Rounds):** Khi tăng từ **80 → 120**, mô hình có thêm cơ hội học các tương tác phức tạp hơn nhưng không bị overfit, nhờ cơ chế *Early Stopping* kích hoạt đúng lúc.

**Điểm sáng:** Các cấu hình tốt đều **hội tụ về cùng một vùng** (`lr 0.05`, `depth 4`, `subsample 1.0`, `colsample 0.8`). Điều này cho thấy không có sự dao động lớn giữa các thử nghiệm – dấu hiệu của một mô hình mạnh và dữ liệu rất phù hợp với thuật toán Boosting.

**Về phần bộ tham số tối ưu**, bộ tham số tốt nhất được xác định là:
 * `n_rounds` = 120
 * `max_depth` = 4
 * `learning_rate` = 0.05
 * `subsample` = 1.0
 * `colsample` = 0.8
 * **Best Threshold** = 0.286
 * **Best F1 (val)** = 0.6261

 **Tại sao bộ tham số này hợp lý?**
  1.  **`learning_rate = 0.05`**: Cho phép mô hình học sâu nhưng vẫn kiểm soát tốt overfitting.
  2.  **`max_depth = 4`**: Đảm bảo mỗi cây không quá phức tạp, tránh việc *fitting* thừa lên các nhiễu (noise) của dữ liệu One-hot.
  3.  **`subsample = 1.0` & `colsample = 0.8`**: Giúp tăng độ đa dạng cây nhưng vẫn giữ được đủ thông tin quan trọng cho mỗi vòng boosting.
  4.  **`threshold = 0.286`**: Phản ánh xác suất dự đoán của XGBoost tương đối "đậm" và được hiệu chỉnh (calibrate) tốt hơn Random Forest. Ngưỡng này cho phép cân bằng tối ưu giữa Precision và Recall.

**Kết luận:**
Quá trình Grid Search khẳng định **XGBoost** học ổn định, không bị Overfit và **liên tục vượt trội** hơn hai mô hình còn lại. Việc các cấu hình tốt hội tụ tại cùng một vùng tham số chứng tỏ dữ liệu rất phù hợp với Boosting và XGBoost đã khai thác thành công bản chất phi tuyến của tập dữ liệu HR.

### **Đánh giá trên test set**

Thực hiện bước đánh giá cuối cùng cho mô hình Champion (XGBoost) với bộ tham số và ngưỡng cắt (threshold) đã tối ưu:

1.  **Áp dụng tham số:** Sử dụng mô hình `best_xgb` và ngưỡng `best_xgb_threshold` (tìm được từ Validation) để dự báo trên tập **Test**.
2.  **Tính toán Metrics:** Tính toàn bộ các chỉ số quan trọng (F1, Recall, Precision) và **ROC-AUC**.
3.  **So sánh đối chứng (Validation vs Test):**
    * Xuất bảng so sánh trực quan để kiểm tra khả năng tổng quát hóa (Generalization).
    * *Mục tiêu:* Đảm bảo điểm số trên Test tương đương Validation (không bị Overfitting).
4.  **Phân tích lỗi:** Hiển thị Ma trận nhầm lẫn (Confusion Matrix) để nhìn rõ số lượng mẫu dự báo đúng/sai thực tế.

In [8]:
print("Best XGBOOST:")
print(f"Best params: n_rounds={best_xgb_params[0]}, "
      f"max_depth={best_xgb_params[1]}, lr={best_xgb_params[2]}, "
      f"subsample={best_xgb_params[3]}, colsample={best_xgb_params[4]}")
print(f"Best F1-score(validation) = {best_xgb_f1:.4f} - threshold = {best_xgb_threshold:.3f}")

y_val_prob_xgb = best_xgb.predict_proba(X_val)
metrics_xgb_val = evaluate_binary_classification(y_true=y_val, y_prob=y_val_prob_xgb, threshold=best_xgb_threshold)

y_test_prob_xgb = best_xgb.predict_proba(X_test)
metrics_xgb_test = evaluate_binary_classification(y_true=y_test, y_prob=y_test_prob_xgb, threshold=best_xgb_threshold)

auc_val_xgb = roc_auc_score(y_val, y_val_prob_xgb)
auc_test_xgb = roc_auc_score(y_test, y_test_prob_xgb)

print("\n" + "="*70)
print("                    XGBoost – Validation vs Test")
print("="*70)
print(f"Best threshold (val): {best_xgb_threshold:.4f}, F1(val) = {best_xgb_f1:.4f}\n")

print(f"{'Metric':15s} | {'Validation':12s} | {'Test':12s}")
print("-"*70)

for k in metrics_xgb_val.keys():
    if k == "confusion_matrix":
        continue

    v_val = metrics_xgb_val[k]
    v_test = metrics_xgb_test[k]

    if isinstance(v_val, float):
        print(f"{k:15s} | {v_val:12.4f} | {v_test:12.4f}")
    else:
        print(f"{k:15s} | {str(v_val):12s} | {str(v_test):12s}")

print(f"{'roc_auc':15s} | {auc_val_xgb:12.4f} | {auc_test_xgb:12.4f}")

print("-"*70)
print("\nConfusion matrix (Validation):")
print(metrics_xgb_val["confusion_matrix"])

print("\nConfusion matrix (Test):")
print(metrics_xgb_test["confusion_matrix"])

Best XGBOOST:
Best params: n_rounds=120, max_depth=4, lr=0.05, subsample=1.0, colsample=0.8
Best F1-score(validation) = 0.6261 - threshold = 0.286

                    XGBoost – Validation vs Test
Best threshold (val): 0.2864, F1(val) = 0.6261

Metric          | Validation   | Test        
----------------------------------------------------------------------
accuracy        |       0.7883 |       0.7873
precision       |       0.5511 |       0.5635
recall          |       0.7247 |       0.7238
f1              |       0.6261 |       0.6337
threshold       |       0.2864 |       0.2864
roc_auc         |       0.7970 |       0.8035
----------------------------------------------------------------------

Confusion matrix (Validation):
[[2341  553]
 [ 258  679]]

Confusion matrix (Test):
[[2311  546]
 [ 269  705]]


#### **Nhận xét**
  1. Độ ổn định tuyệt vời 

        Mô hình XGBoost thể hiện sự ổn định cao nhất trong cả 3 mô hình, chứng tỏ khả năng tổng quát hóa (generalization) xuất sắc:

        | Metric | Val | Test | Nhận xét |
        | :--- | :---: | :---: | :--- |
        | **Accuracy** | 0.7883 | **0.7873** | Gần như giống nhau |
        | **Precision** | 0.5511 | **0.5635** | Tăng nhẹ |
        | **Recall** | 0.7247 | **0.7238** | Giữ nguyên (Giảm không đáng kể) |
        | **F1-Score** | 0.6261 | **0.6337** | Tăng lên → Tín hiệu rất tốt |
        | **ROC-AUC** | 0.7970 | **0.8035** | Cao nhất trong 3 mô hình |

        **Kết luận:** Không có dấu hiệu Overfitting. Mô hình hoạt động tin cậy trên tập dữ liệu chưa từng thấy (Test set).

  2. Dẫn đầu về F1-Score
        
        So sánh hiệu năng tổng thể trên tập Test:
        * **Logistic Regression:** 0.6151
        * **Random Forest:** 0.6109
        * **XGBoost:** **0.6337**

        **XGBoost dẫn đầu**, khẳng định vị thế vượt trội của thuật toán Gradient Boosting trên dữ liệu dạng bảng (Tabular Data).

  3. Điểm cân bằng hoàn hảo (Precision vs Recall)
        | Metric | Logistic | RF | XGBoost |
        | :--- | :---: | :---: | :---: |
        | **Precision** | 0.5267 | 0.5555 |  **0.5635** (Cao nhất) |
        | **Recall** | 0.7392 | 0.6786 | **0.7238** (Tiệm cận Logistic) |

        **Phân tích:**
        * **Precision cao nhất:** Giảm thiểu số lượng báo động giả (False Positive) tốt nhất.
        * **Recall ấn tượng:** Gần ngang ngửa Logistic, nghĩa là không bỏ sót nhiều nhân tài muốn ra đi.
        * Đây là sự cân bằng tối ưu mà chúng ta tìm kiếm cho bài toán quản trị nhân sự.

  4. Ngưỡng cắt (Threshold) hợp lý & Phân loại tốt
        * **Threshold tối ưu:** `0.2864`.
            * So sánh: Logistic (0.266) < XGBoost (0.286) < RF (0.03).
            * **Nhận định:** Xác suất dự báo của XGBoost được hiệu chỉnh (calibrated) tốt hơn hẳn Random Forest, gần với phân phối thực tế hơn.
        * **ROC-AUC (0.8035):** Cao nhất trong 3 mô hình, chứng tỏ khả năng phân biệt giữa hai lớp (Nghỉ việc vs Ở lại) là tốt nhất.

  5. Phân tích Ma trận nhầm lẫn (Test Set)
     Kết quả:
        ```text
        [[2311  546]
        [ 269  705]]
        ```
     * **False Positive (546) - Báo động giả:**
         * Kết quả nằm giữa Logistic (647) và Random Forest (529).
         * *Nhận định:* Đây là mức "hy sinh" chấp nhận được về mặt độ chính xác (Precision) để đổi lấy khả năng bao quát (Recall) tốt hơn.

     * **False Negative (269) - Bỏ sót rủi ro:**
         * Thấp hơn đáng kể so với Random Forest (313) và chỉ nhỉnh hơn Logistic một chút (254).
         * *Nhận định:* Mô hình hạn chế tối đa việc bỏ sót những nhân sự muốn nghỉ việc, đảm bảo tính an toàn cho kế hoạch nhân sự.

**Tổng kết: XGBoost là mô hình tốt nhất** vì đã giải quyết xuất sắc bài toán **Cân bằng (Trade-off)** mà hai mô hình trước gặp phải:
  1. **FN không quá cao như Random Forest** (giữ được Recall tốt).
  2. **FP không quá cao như Logistic Regression** (giữ được Precision ổn).
  3. **Kết quả:** Đạt **F1-Score cao nhất**, trở thành mô hình toàn diện nhất để đưa vào áp dụng thực tế.

## **So sánh & Tổng kết**
- Tổng hợp Accuracy, Precision, Recall, F1 trên cùng tập Validation.
- Dựa vào metric ưu tiên (ở đây là F1), chọn ra best model.
- Nhận xét và phân tích kết quả.

In [9]:
def summarize(name, metrics):
    return {
        "model": name,
        "accuracy": metrics["accuracy"],
        "precision": metrics["precision"],
        "recall": metrics["recall"],
        "f1": metrics["f1"],
    }

rows_test = [
    summarize("LogisticRegression", metrics_log_test),
    summarize("RandomForest",       metrics_rf_test_best),
    summarize("XGBoost",            metrics_xgb_test),
]

best_model = max(rows_test, key=lambda r: r["f1"])
print("="*65)
print("\t\t\tSo sánh các model")
print("="*65)

print(f"{'Model':18s} | {'Accuracy':9s} | {'Precision':9s} | {'Recall':9s} | {'F1-score':9s}")
print("-"*65)

for r in rows_test:
    print(f"{r['model']:18s} | {r['accuracy']:.4f}     | {r['precision']:.4f}     | {r['recall']:.4f}     | {r['f1']:.4f}")

print("-"*65)
print(f"Best model theo F1 trên TEST: {best_model['model']} (F1={best_model['f1']:.4f})")
print("="*65)

			So sánh các model
Model              | Accuracy  | Precision | Recall    | F1-score 
-----------------------------------------------------------------
LogisticRegression | 0.7648     | 0.5267     | 0.7392     | 0.6151
RandomForest       | 0.7802     | 0.5555     | 0.6786     | 0.6109
XGBoost            | 0.7873     | 0.5635     | 0.7238     | 0.6337
-----------------------------------------------------------------
Best model theo F1 trên TEST: XGBoost (F1=0.6337)


### **Phân tích và nhận xét kết luận**

Bảng kết quả thực nghiệm trên bộ dữ liệu **TEST** đã vẽ nên bức tranh rõ ràng về hành vi của ba mô hình (Logistic Regression, Random Forest, XGBoost). Dưới đây là những nhận định chi tiết về mức độ phù hợp của từng thuật toán đối với bài toán HR Analytics:

#### 1. Logistic Regression (Baseline mạnh về Recall)
Mặc dù là mô hình tuyến tính đơn giản, Logistic Regression thể hiện hiệu năng khá ấn tượng:
* **Ưu điểm:** Đạt **Recall cao nhất (0.7392)** trong cả ba mô hình. Điều này đồng nghĩa với việc mô hình rất giỏi trong việc "vơ vét", ít bỏ sót các ứng viên thực sự muốn đổi việc.
* **Nhược điểm:** **Precision còn thấp (0.5267)**, tức là mô hình dự báo nhầm khá nhiều người ở lại thành người đi (False Positive cao).
* **Đánh giá:** Đóng vai trò là một Baseline ổn định với độ chính xác (**Accuracy**) đạt **0.7648**. Tuy nhiên, chỉ số **F1 (0.6151)** cho thấy sự cân bằng giữa Precision và Recall chưa thực sự tối ưu.

#### 2. Random Forest (Sự đánh đổi về Precision)
Random Forest mang lại một cách tiếp cận khác biệt so với Logistic Regression:
* **Cải thiện:** Tăng được **Precision lên 0.5555**, giúp giảm bớt số lượng cảnh báo giả ("đỡ đoán nhầm" hơn).
* **Sụt giảm:** Đổi lại, **Recall giảm xuống còn 0.6786**, dẫn đến việc bỏ sót nhiều trường hợp rủi ro hơn.
* **Đánh giá:** Với **F1 đạt 0.6109** (thấp hơn cả Logistic), RF cho thấy khả năng cân bằng kém hơn trong bài toán này. Nguyên nhân có thể do hạn chế của RF khi xử lý dữ liệu thưa (One-hot encoding nhiều chiều) và giới hạn về độ sâu của cây.

#### 3. XGBoost (The best model - Sự cân bằng hoàn hảo) 
XGBoost khẳng định vị thế vượt trội trên mọi phương diện quan trọng:
* **Hiệu suất:** Đạt **Precision cao nhất (0.5635)** và **Accuracy cao nhất (0.7873)**.
* **Khả năng bao quát:** Giữ được **Recall rất cao (0.7238)**, tiệm cận sát nút với Logistic Regression.
* **Điểm quyết định:** Chỉ số **F1 đạt 0.6337** (cao nhất trong 3 mô hình).
* **Đánh giá:** XGBoost là mô hình tốt nhất vì nó giải quyết được bài toán đánh đổi: vừa bắt được đúng người tìm việc (High Recall), vừa giảm thiểu báo động giả (High Precision).

#### 4. Nghịch lý Mô hình (Model Paradox)
**1. Tại sao Random Forest (Phi tuyến) thua Logistic Regression (Tuyến tính)?**

Mặc dù dữ liệu có tính phi tuyến, Random Forest (RF) hoạt động kém hiệu quả do:
* **Vấn đề One-Hot Encoding:** RF gặp khó khăn khi phân chia trên ma trận thưa (nhiều cột 0/1), dẫn đến việc các cây con bị yếu đi do chọn phải đặc trưng kém quan trọng.
* **Feature Engineering hiệu quả:** Việc chúng ta tạo sẵn các biến tương tác (như `brain_drain_risk`) và làm sạch dữ liệu kỹ lưỡng đã giúp Logistic Regression dễ dàng tiếp cận các mối quan hệ phức tạp dưới dạng tuyến tính hóa, tận dụng tối đa khả năng tối ưu trọng số toàn cục của nó.

**2. Tại sao XGBoost chiến thắng tuyệt đối (dù cùng họ cây với RF)?**

XGBoost vượt trội nhờ cơ chế **Gradient Boosting**:
* **Học từ lỗi sai:** Thay vì bỏ phiếu ngang hàng như RF, XGBoost xây cây tuần tự để khắc phục lỗi của các cây trước đó, giúp nó bắt được các mẫu dữ liệu khó (hard cases) mà RF bỏ qua.
* **Tối ưu hóa trực tiếp:** XGBoost tối ưu hóa hàm mất mát (Log-Loss) tương tự Logistic Regression nhưng trên không gian phi tuyến, kết hợp sức mạnh của cả hai thế giới.
* **Sparsity Aware:** XGBoost xử lý dữ liệu One-Hot Encoding hiệu quả hơn hẳn thuật toán chia nút truyền thống của Random Forest.

#### **Kết luận chung**
Trong ba mô hình, **XGBoost** là lựa chọn tối ưu nhất nhờ khả năng cân bằng xuất sắc giữa Precision và Recall. Kết quả này phản ánh đúng đặc tính của dữ liệu dạng bảng (Structured/Tabular Data), nơi các thuật toán Gradient Boosting như XGBoost thường xuyên chiếm ưu thế so với các phương pháp truyền thống.
